In [13]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict

In [14]:
torch.set_grad_enabled(False)

In [15]:
def get_x(shape= (4,3,32,224,224)):
    x_pt = torch.rand(shape) * 255
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

In [16]:
shape_of_input = [10,3,32,224,224]
x_tf, x_pt = get_x(shape_of_input)

## Convert Weights

Initiate model and Load PyTorch Weights

In [17]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

In [18]:
# shape_of_input[1] , shape_of_input[4], shape_of_input[2] = shape_of_input[2], shape_of_input[1], shape_of_input[4]

# (2, 32, 224,224,3)
# shape_of_input = [1,3,32,224,224]

print("shape_of_input", shape_of_input)
print("pt")
pt_model = SwinTransformer3D_pt(**cfg,drop_rate=0.4, drop_path_rate=0., isTest= True)
print("++++++++\n tf")
tf_model = SwinTransformer3D(**cfg,shape_of_input=shape_of_input, drop_rate=0.4, drop_path_rate=0., isTest= True)
x_tf, x_pt = get_x()


print("\n\n\n pt")
basic_pt, z= pt_model(x_pt)
print("------- \n tf")

basic_tf, y = tf_model(x_tf)

# print("\n loading checkpoint")
# checkpoint = torch.load(f'{name}.pth')
# new_state_dict = OrderedDict()
# for k, v in checkpoint['state_dict'].items():
#     if 'backbone' in k:
#         name = k[9:]
#         new_state_dict[name] = v 

# pt_model.load_state_dict(new_state_dict) 
# pt_model.eval()

shape_of_input [10, 3, 32, 224, 224]
pt
++++++++
 tf
shape_of_input:  [10, 3, 32, 224, 224]



 pt
------- 
 tf
patch embed (4, 96, 16, 56, 56)


In [19]:
# a = a

In [20]:
pt_model.eval()


Convert Functions

In [21]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


### convert

In [22]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [23]:
x_tf, x_pt = get_x(shape_of_input)

In [24]:
layers_output_tf, y = tf_model(x_tf)
print("------")
layers_output_pt, z= pt_model(x_pt)

y.shape, z.shape

# basic 1 input (1, 96, 4, 56, 56)

patch embed (10, 96, 16, 56, 56)
------


(TensorShape([10, 768, 16, 7, 7]), torch.Size([10, 768, 16, 7, 7]))

In [25]:
# Eplore the output of all layers
for layer in layers_output_pt:
    print("--------------",layer, "-------------\n TF: ", layers_output_tf[layer].numpy()[:1,:1,:1,:1,:10], "\n PT: ", layers_output_pt[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n" )

-------------- PatchEmbed -------------
 TF:  [[[[[0.34746313 0.00200087 1.4394778  1.7740839  0.24191917 0.16951421
     0.5117054  0.08692615 0.17793417 0.62967354]]]]] 
 PT:  [[[[[0.3474632  0.00200081 1.4394777  1.7740839  0.24191923 0.16951421
     0.51170534 0.08692621 0.1779342  0.6296734 ]]]]] 

-------------- drop_out -------------
 TF:  [[[[[0.34746313 0.00200087 1.4394778  1.7740839  0.24191917 0.16951421
     0.5117054  0.08692615 0.17793417 0.62967354]]]]] 
 PT:  [[[[[0.3474632  0.00200081 1.4394777  1.7740839  0.24191923 0.16951421
     0.51170534 0.08692621 0.1779342  0.6296734 ]]]]] 

-------------- basic layer1 -------------
 TF:  [[[[[0.74382645 1.0854849  0.45899203 0.4616804  0.9776213  1.3806359
     0.3153539  0.4985238  1.026734   0.7362664 ]]]]] 
 PT:  [[[[[0.74382627 1.0854849  0.45899203 0.46168035 0.97762173 1.3806362
     0.31535393 0.498524   1.0267339  0.7362657 ]]]]] 

-------------- basic layer2 -------------
 TF:  [[[[[ 0.17765006 -0.26607105 -0.2244483

In [26]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:

    abs_diff = np.sum( np.abs( layers_output_tf[layer].numpy() - layers_output_pt[layer].detach().numpy()))
    print(f"{abs_diff} Absolute Difference of  {layer}" )


8.119380950927734 Absolute Difference of  PatchEmbed
8.119380950927734 Absolute Difference of  drop_out
4.743508815765381 Absolute Difference of  basic layer1
3.7834949493408203 Absolute Difference of  basic layer2
2.67332124710083 Absolute Difference of  basic layer3
3.5226778984069824 Absolute Difference of  basic layer4
4.604726791381836 Absolute Difference of  Final Output


In [29]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:
    print("Testing", layer)
    np.testing.assert_allclose(layers_output_tf[layer].numpy(), layers_output_pt[layer].detach().numpy(), 1e-4, 1e-4)

Testing PatchEmbed
Testing drop_out
Testing basic layer1
Testing basic layer2
Testing basic layer3
Testing basic layer4
Testing Final Output


In [28]:
a = a

NameError: name 'a' is not defined

In [ ]:
e = 1e-4
np.testing.assert_allclose(layers_output_tf["basic layer4"].numpy(), layers_output_pt["basic layer4"].detach().numpy(), e, e)


In [ ]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-4, 1e-4)


### PT basic layer outputs comparison

In [ ]:
attempts = 2
# x_pt = torch.rand((1,3,8,224,224))

outputs = []

for i in range(attempts):
    layer_out , result = pt_model(x_pt)
    outputs.append(layer_out)

i = 0
for layer in outputs[0]:
    print("--------------", layer, "---------------")
    for idx, layer_out in enumerate(outputs) :
        print(f"attempt {idx} : ",layer_out[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n")
    print()


### All close testing

Compare the first attempt with another attempt. Enter the attempt value in the following cell

In [ ]:
attempt_no = 1     # Change the attempt_no value to compare the first attempt with another attempt


In [ ]:
# PatchEmbed Layer
output1 = outputs[0]["PatchEmbed"]
output2 =  outputs[attempt_no]["PatchEmbed"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer1
output1 = outputs[0]["basic layer1"]
output2 =  outputs[attempt_no]["basic layer1"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer2

output1 = outputs[0]["basic layer2"]
output2 =  outputs[attempt_no]["basic layer2"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer3

output1 = outputs[0]["basic layer3"]
output2 =  outputs[attempt_no]["basic layer3"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Basic Layer4

output1 = outputs[0]["basic layer4"]
output2 =  outputs[attempt_no]["basic layer4"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [ ]:
# Final Output

output1 = outputs[0]["Final Output"]
output2 =  outputs[attempt_no]["Final Output"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)